In [1]:
# --- Cell purpose: Import core libraries ---
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
import requests
import warnings
warnings.filterwarnings("ignore")

print("✅ Core libraries imported")


✅ Core libraries imported


In [2]:
# --- Cell purpose: Import visualization libraries ---
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import gradio as gr
from PIL import Image
import io

print("✅ Visualization libraries imported")


✅ Visualization libraries imported


In [ ]:
# --- Cell purpose: Hugging Face API configuration ---
# Your HF token
HF_TOKEN = "key"

# API base URL
HF_API_BASE = "https://api-inference.huggingface.co/models"

# Model configurations for APIs
API_MODELS = {
    "text_generation": "microsoft/DialoGPT-large",
    "classification": "facebook/bart-large-mnli",
    "summarization": "facebook/bart-large-cnn",
    "image_generation": "stabilityai/stable-diffusion-2-1",
    "speech_to_text": "openai/whisper-small",
    "astronomy_llm": "microsoft/DialoGPT-large",
    "geoscience_llm": "microsoft/DialoGPT-large"
}

headers = {"Authorization": f"Bearer {HF_TOKEN}"}

def hf_api_call(model_name, payload, is_image=False):
    """Universal HF API caller"""
    url = f"{HF_API_BASE}/{model_name}"
    
    if is_image:
        response = requests.post(url, headers=headers, json=payload)
        if response.status_code == 200:
            return response.content
        return None
    else:
        response = requests.post(url, headers=headers, json=payload)
        return response.json()

print("✅ Hugging Face API configured")


✅ Hugging Face API configured


In [4]:
# --- Cell purpose: API-based query classifier ---
class QueryClassificationAgent:
    def __init__(self):
        self.model = API_MODELS["classification"]
    
    def classify_query(self, query):
        """Classify user query into categories"""
        try:
            payload = {
                "inputs": query,
                "parameters": {
                    "candidate_labels": ["astronomy", "geoscience", "both", "general"]
                }
            }
            
            result = hf_api_call(self.model, payload)
            
            if isinstance(result, dict) and "labels" in result:
                return result["labels"][0]
            
        except Exception as e:
            print(f"Classification error: {e}")
        
        # Fallback keyword classification
        query_lower = query.lower()
        astronomy_keywords = ["star", "planet", "space", "galaxy", "mars", "jupiter", "satellite", "astronaut"]
        geoscience_keywords = ["earth", "weather", "climate", "earthquake", "ocean", "geology", "rain", "temperature"]
        
        astro_score = sum(1 for word in astronomy_keywords if word in query_lower)
        geo_score = sum(1 for word in geoscience_keywords if word in query_lower)
        
        if astro_score > 0 and geo_score > 0:
            return "both"
        elif astro_score > 0:
            return "astronomy"
        elif geo_score > 0:
            return "geoscience"
        else:
            return "general"

query_classifier = QueryClassificationAgent()
print("✅ Query classifier ready")


✅ Query classifier ready


In [5]:
# --- Cell purpose: API-based knowledge generator ---
class KnowledgeAgent:
    def __init__(self):
        self.model = API_MODELS["text_generation"]
        self.knowledge_base = {
            "astronomy": {
                "stars": "Stars are massive, luminous spheres of plasma held together by gravity, generating energy through nuclear fusion.",
                "planets": "Planets are celestial bodies that orbit stars and have cleared their orbital paths of other debris.",
                "galaxies": "Galaxies are massive collections of stars, gas, dust, and dark matter held together by gravity.",
                "black_holes": "Black holes are regions of spacetime with gravitational effects so strong that nothing can escape."
            },
            "geoscience": {
                "weather": "Weather is the state of the atmosphere at a specific time and place, including temperature, humidity, and precipitation.",
                "earthquakes": "Earthquakes occur due to sudden movements of tectonic plates beneath Earth's surface.",
                "climate": "Climate is the long-term average of weather patterns in a particular region over decades.",
                "satellites": "Earth observation satellites monitor weather, climate, natural disasters, and environmental changes."
            }
        }
    
    def generate_response(self, query, category="general"):
        """Generate comprehensive response using API + knowledge base"""
        
        # Get relevant context from knowledge base
        context = self.get_relevant_context(query, category)
        
        # Create enhanced prompt
        prompt = f"""Context: {context}

Question: {query}

Provide a detailed, educational response about this topic. Include scientific facts and explain concepts clearly."""

        try:
            payload = {
                "inputs": prompt,
                "parameters": {
                    "max_new_tokens": 300,
                    "temperature": 0.7,
                    "return_full_text": False
                }
            }
            
            result = hf_api_call(self.model, payload)
            
            if isinstance(result, list) and len(result) > 0:
                generated = result[0].get("generated_text", "")
                # Combine context + generated response
                return f"{context}\n\n{generated}"
            elif isinstance(result, dict):
                generated = result.get("generated_text", "")
                return f"{context}\n\n{generated}"
                
        except Exception as e:
            print(f"Generation error: {e}")
        
        # Fallback to knowledge base only
        return context
    
    def get_relevant_context(self, query, category):
        """Get relevant information from knowledge base"""
        query_lower = query.lower()
        contexts = []
        
        if category in ["astronomy", "both"]:
            for topic, info in self.knowledge_base["astronomy"].items():
                if topic in query_lower:
                    contexts.append(info)
        
        if category in ["geoscience", "both"]:
            for topic, info in self.knowledge_base["geoscience"].items():
                if topic in query_lower:
                    contexts.append(info)
        
        if not contexts:
            # Default informative response based on category
            if "astronomy" in category:
                contexts.append("Astronomy is the scientific study of celestial objects and phenomena beyond Earth's atmosphere.")
            elif "geoscience" in category:
                contexts.append("Geoscience encompasses the study of Earth's physical structure, processes, and history.")
            else:
                contexts.append("This question relates to scientific understanding of our universe and planet.")
        
        return " ".join(contexts)

knowledge_agent = KnowledgeAgent()
print("✅ Knowledge agent ready")


✅ Knowledge agent ready


In [6]:
# --- Cell purpose: API-based image generator ---
class ImageGenerationAgent:
    def __init__(self):
        self.model = API_MODELS["image_generation"]
    
    def generate_image(self, prompt, category="general"):
        """Generate image using HF API"""
        
        # Enhance prompt based on category
        if category == "astronomy":
            enhanced_prompt = f"space astronomy {prompt}, cosmic, stars, planets, detailed scientific illustration"
        elif category == "geoscience":
            enhanced_prompt = f"earth science {prompt}, satellite view, environmental, geological, detailed scientific visualization"
        else:
            enhanced_prompt = f"scientific illustration {prompt}, detailed, educational"
        
        try:
            payload = {"inputs": enhanced_prompt}
            image_data = hf_api_call(self.model, payload, is_image=True)
            
            if image_data:
                # Convert bytes to PIL Image
                image = Image.open(io.BytesIO(image_data))
                return image
                
        except Exception as e:
            print(f"Image generation error: {e}")
        
        return None

image_generator = ImageGenerationAgent()
print("✅ Image generator ready")


✅ Image generator ready


In [7]:
# --- Cell purpose: External data fetcher ---
class DataFetchAgent:
    def __init__(self):
        self.apis = {
            "nasa_apod": "https://api.nasa.gov/planetary/apod",
            "nasa_neo": "https://api.nasa.gov/neo/rest/v1/feed",
            "weather": "https://api.openweathermap.org/data/2.5/weather"
        }
    
    def fetch_nasa_apod(self, api_key="DEMO_KEY"):
        """Fetch NASA Astronomy Picture of the Day"""
        try:
            response = requests.get(f"{self.apis['nasa_apod']}?api_key={api_key}")
            if response.status_code == 200:
                return response.json()
        except Exception as e:
            print(f"NASA APOD error: {e}")
        
        return {
            "title": "Sample Space Image", 
            "explanation": "This is a demo entry. Get NASA API key for real data.",
            "url": "https://example.com/space.jpg"
        }
    
    def fetch_weather_data(self, city="Mumbai"):
        """Fetch weather data (demo)"""
        # This is a placeholder - integrate with real weather APIs
        return {
            "city": city,
            "temperature": "28°C",
            "humidity": "75%",
            "conditions": "Partly Cloudy",
            "note": "Demo data - integrate with OpenWeatherMap or MOSDAC APIs"
        }
    
    def fetch_space_weather(self):
        """Fetch space weather data (demo)"""
        return {
            "solar_wind_speed": "450 km/s",
            "magnetic_field": "5.2 nT",
            "aurora_activity": "Moderate",
            "note": "Demo data - integrate with NOAA Space Weather APIs"
        }

data_fetcher = DataFetchAgent()
print("✅ Data fetcher ready")


✅ Data fetcher ready


In [8]:
# --- Cell purpose: Astronomy specialist ---
class AstronomyAgent:
    def __init__(self, knowledge_agent, data_fetcher, image_generator):
        self.knowledge_agent = knowledge_agent
        self.data_fetcher = data_fetcher
        self.image_generator = image_generator
    
    def process_query(self, query):
        """Process astronomy-specific queries"""
        
        # Generate knowledge response
        response = self.knowledge_agent.generate_response(query, "astronomy")
        
        result = {
            "category": "astronomy",
            "text_response": response,
            "data": {},
            "image": None
        }
        
        # Fetch relevant astronomical data
        if any(word in query.lower() for word in ["nasa", "apod", "picture", "today"]):
            nasa_data = self.data_fetcher.fetch_nasa_apod()
            result["data"]["nasa_apod"] = nasa_data
        
        if any(word in query.lower() for word in ["space weather", "solar", "aurora"]):
            space_weather = self.data_fetcher.fetch_space_weather()
            result["data"]["space_weather"] = space_weather
        
        # Generate image if requested
        if any(word in query.lower() for word in ["show", "image", "visualize", "picture"]):
            image = self.image_generator.generate_image(query, "astronomy")
            result["image"] = image
        
        return result

astronomy_agent = AstronomyAgent(knowledge_agent, data_fetcher, image_generator)
print("✅ Astronomy agent ready")


✅ Astronomy agent ready


In [9]:
# --- Cell purpose: Geoscience specialist ---
class GeoscienceAgent:
    def __init__(self, knowledge_agent, data_fetcher, image_generator):
        self.knowledge_agent = knowledge_agent
        self.data_fetcher = data_fetcher
        self.image_generator = image_generator
    
    def process_query(self, query):
        """Process geoscience-specific queries"""
        
        # Generate knowledge response
        response = self.knowledge_agent.generate_response(query, "geoscience")
        
        result = {
            "category": "geoscience", 
            "text_response": response,
            "data": {},
            "image": None
        }
        
        # Fetch relevant Earth science data
        if any(word in query.lower() for word in ["weather", "temperature", "climate"]):
            weather_data = self.data_fetcher.fetch_weather_data()
            result["data"]["weather"] = weather_data
        
        # Generate satellite/earth imagery if requested
        if any(word in query.lower() for word in ["satellite", "map", "earth", "visualize"]):
            image = self.image_generator.generate_image(query, "geoscience")
            result["image"] = image
        
        return result

geoscience_agent = GeoscienceAgent(knowledge_agent, data_fetcher, image_generator)
print("✅ Geoscience agent ready")


✅ Geoscience agent ready


In [10]:
# --- Cell purpose: Main system orchestrator ---
class AstroGeoOrchestrator:
    def __init__(self):
        self.classifier = query_classifier
        self.astronomy_agent = astronomy_agent
        self.geoscience_agent = geoscience_agent
        self.knowledge_agent = knowledge_agent
        
    def process_query(self, query):
        """Main query processing pipeline"""
        
        if not query or len(query.strip()) < 3:
            return {
                "category": "error",
                "text_response": "Please provide a valid question about astronomy or geoscience.",
                "data": {},
                "image": None
            }
        
        # Step 1: Classify the query
        category = self.classifier.classify_query(query)
        
        # Step 2: Route to appropriate agent
        if category == "astronomy":
            result = self.astronomy_agent.process_query(query)
        elif category == "geoscience":
            result = self.geoscience_agent.process_query(query)
        elif category == "both":
            # Handle interdisciplinary queries
            astro_result = self.astronomy_agent.process_query(query)
            geo_result = self.geoscience_agent.process_query(query)
            
            result = {
                "category": "both",
                "text_response": f"**Astronomy Perspective:**\n{astro_result['text_response']}\n\n**Geoscience Perspective:**\n{geo_result['text_response']}",
                "data": {**astro_result.get("data", {}), **geo_result.get("data", {})},
                "image": astro_result.get("image") or geo_result.get("image")
            }
        else:
            # General scientific query
            response = self.knowledge_agent.generate_response(query, "general")
            result = {
                "category": "general",
                "text_response": response,
                "data": {},
                "image": None
            }
        
        return result

orchestrator = AstroGeoOrchestrator()
print("✅ Main orchestrator ready")


✅ Main orchestrator ready


In [11]:
# --- Cell purpose: User interface ---
def create_astrogeo_interface():
    with gr.Blocks(
        title="🌌 AstroGeo Multi-Agent AI",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            font-family: 'Segoe UI', system-ui, sans-serif;
        }
        .main-header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 25px;
            border-radius: 15px;
            margin-bottom: 25px;
            text-align: center;
        }
        """
    ) as demo:
        
        # Header
        gr.HTML("""
        <div class="main-header">
            <h1>🌌 AstroGeo Multi-Agent AI System</h1>
            <p>Advanced Astronomy & Geoscience Intelligence</p>
            <p><strong>Powered by Hugging Face APIs • Multi-Agent Architecture • Real-time Data</strong></p>
        </div>
        """)
        
        with gr.Row():
            with gr.Column(scale=2):
                # Input section
                query_input = gr.Textbox(
                    placeholder="Ask about stars, planets, weather, earthquakes, satellites, climate...",
                    label="🔍 Your Question",
                    lines=2
                )
                
                with gr.Row():
                    submit_btn = gr.Button("🚀 Ask AstroGeo AI", variant="primary", size="lg")
                    clear_btn = gr.Button("🗑️ Clear", variant="secondary")
                
                # Advanced options
                with gr.Accordion("🔧 Advanced Options", open=False):
                    include_image = gr.Checkbox(label="Generate visualization", value=True)
                    include_data = gr.Checkbox(label="Fetch real-time data", value=True)
                    response_length = gr.Slider(1, 5, value=3, label="Response detail level")
            
            with gr.Column(scale=3):
                # Output section
                category_output = gr.Textbox(label="🏷️ Query Category", interactive=False)
                response_output = gr.Textbox(label="🤖 AI Response", lines=12)
                
        with gr.Row():
            with gr.Column():
                image_output = gr.Image(label="🎨 Generated Visualization", height=300)
            with gr.Column():
                data_output = gr.JSON(label="📊 Real-time Data", height=300)
        
        # Event handlers
        def process_query(query, gen_image, fetch_data, detail_level):
            if not query:
                return "", "Please enter a question", {}, None
            
            # Process the query
            result = orchestrator.process_query(query)
            
            # Prepare outputs
            category = result.get("category", "Unknown")
            response = result.get("text_response", "No response generated")
            data = result.get("data", {}) if fetch_data else {}
            image = result.get("image") if gen_image else None
            
            return category, response, data, image
        
        def clear_all():
            return "", "", "", {}, None
        
        # Connect events
        submit_btn.click(
            process_query,
            inputs=[query_input, include_image, include_data, response_length],
            outputs=[category_output, response_output, data_output, image_output]
        )
        
        clear_btn.click(
            clear_all,
            outputs=[query_input, category_output, response_output, data_output, image_output]
        )
        
        # Example queries
        gr.Examples(
            examples=[
                "What are black holes and how do they form?",
                "Explain the water cycle and climate change",
                "Show me a visualization of Mars",
                "How do satellites monitor weather?",
                "What causes earthquakes?",
                "Visualize the solar system",
                "Today's space weather conditions"
            ],
            inputs=query_input
        )
    
    return demo

# Create interface
astrogeo_demo = create_astrogeo_interface()
print("✅ AstroGeo interface created")


✅ AstroGeo interface created


In [ ]:
# --- Cell purpose: Launch the application ---
def launch_astrogeo():
    print("🚀 Launching AstroGeo Multi-Agent AI System...")
    print("=" * 50)
    print("Features:")
    print("✅ 100% Hugging Face APIs")
    print("✅ Multi-agent architecture")
    print("✅ Real-time data integration")
    print("✅ Image generation")
    print("✅ Advanced query processing")
    print("=" * 50)
    
    astrogeo_demo.launch(
        server_port=7860,
        share=False,  # Set to True for public sharing
        debug=True,
        inbrowser=True
    )

# Launch the system
if __name__ == "__main__":
    launch_astrogeo()


🚀 Launching AstroGeo Multi-Agent AI System...
Features:
✅ 100% Hugging Face APIs
✅ Multi-agent architecture
✅ Real-time data integration
✅ Image generation
✅ Advanced query processing
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Generation error: Expecting value: line 1 column 1 (char 0)


In [ ]:
# --- Cell purpose: System testing ---
def test_astrogeo_system():
    """Test all components"""
    print("🧪 Testing AstroGeo System...")
    print("=" * 40)
    
    test_queries = [
        "What are stars?",
        "How do satellites work?", 
        "Show me Mars",
        "What causes rain?"
    ]
    
    for i, query in enumerate(test_queries, 1):
        print(f"\nTest {i}: {query}")
        result = orchestrator.process_query(query)
        print(f"Category: {result['category']}")
        print(f"Response: {result['text_response'][:100]}...")
        print("✅ Test passed")
    
    print("\n🎉 All tests completed successfully!")

# Run tests
test_astrogeo_system()
